In [4]:
import torch
import matplotlib.pyplot as plt
from models import MemNet
from LaMemDataset import LaMemEvalDataset
from torchvision import transforms
import numpy as np
import PIL.Image
from PIL import Image
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
from matplotlib.colors import LinearSegmentedColormap

import pandas as pd

import torchvision.models as Alexnet
from torchvision import transforms

import torch
import torchvision.models as models

from scipy import stats

In [5]:
# Step 1: Load the pretrained AlexNet model
model = models.alexnet(pretrained=True)
model.eval()


/Users/ramahuja/anaconda3/envs/feature/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ramahuja/anaconda3/envs/feature/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [10]:
import json
import requests
# Step 2: Define image preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Step 3: Load and preprocess the image
img = Image.open("/Users/ramahuja/Library/CloudStorage/Dropbox-KoLab/ViTA Lab Datastore/users/ram/thesis/images/coco/coco200/im24.png")  # Replace with your image path
img_t = preprocess(img)
batch_t = torch.unsqueeze(img_t, 0)

# Step 4: Perform inference
with torch.no_grad():
    out = model(batch_t)

url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(url)
labels = response.json()


# Step 6: Get the top 3 predictions
_, indices = torch.sort(out, descending=True)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100

for i in range(3):
    idx = indices[0][i]
    print(f"{labels[idx]}: {percentage[idx].item():.2f}%")

African bush elephant: 59.75%
tusker: 22.98%
Asian elephant: 14.21%


In [11]:
#MEMNET
model_memnet = MemNet()
checkpoint_memnet = torch.utils.model_zoo.load_url("https://github.com/andrewrkeyes/Memnet-Pytorch-Model/raw/master/model.ckpt")
model_memnet.load_state_dict(checkpoint_memnet["state_dict"])

<All keys matched successfully>

In [12]:
# Step 4: Perform inference
with torch.no_grad():
    memorability_score = model_memnet(batch_t)

print(f"Memorability Score: {memorability_score.item()}")


Memorability Score: 0.7394089102745056
